# Online musical genres scrapper

This notebook retrieve musical genres from an online html page and store them in a json file

In [136]:
import urllib.request
from urllib.request import urlopen
import html2text
import codecs
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
    
import requests
from bs4 import BeautifulSoup

In [2]:
from urllib.request import Request, urlopen

#check the url to upload
req = Request('https://en.wikipedia.org/wiki/List_of_music_styles', headers={'User-Agent': 'Chrome/73.0'})
webpage = urlopen(req).read()

In [5]:
webpage

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of music styles - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_music_styles","wgTitle":"List of music styles","wgCurRevisionId":887779608,"wgRevisionId":887779608,"wgArticleId":214666,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Popular music","Lists of music genres"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","A

In [155]:
def get_music_genres(url):
    """
    This function reads the given url, 
    read the html file, parse the information and return a corresponding dictionary

    Params
    ------
    url : str
        url from which the data has to be retrieved

    Returns
    -------
    dictionary
        Returns a dictionary of genres
    """
    req = Request(url, headers={'User-Agent': 'Chrome/73.0'})
    try:
        urlopen(req)
    except:
        print("rejected")
        return ""
    
    soup = BeautifulSoup(urlopen(req).read())
    
    dict_ = {}
    
    #Get all main genre naims first, set them as dictionary key
    main_titles = soup.find_all('span', attrs={'class':'toctext'})
    counter = 0
    print(main_titles)
    for span_ in main_titles:
        if counter > 4:
            key = next(span_.stripped_strings).lower()
            dict_[key] = ''
        counter+=1
        
    #Remove unnecessary keys
    dict_.pop('comedy')
    dict_.pop('classical music')
    dict_.pop('other')
    dict_.pop('references')
    dict_.pop('external links')
    dict_.pop('bibliography')
    dict_.pop('see also')
    
    #find all listed sub-genres
    all_ul = soup.find_all('div', attrs={'class':'div-col columns column-width'})
    result = {}
    j= 0
    
    #iterate through the list of all listed sub-genres
    for i in range(1, len(all_ul)):
        sub_genres_ = []
        for ul_ in all_ul[i].find_all("ul"):
            for li_ in ul_.find_all("li"):
                value = next(li_.stripped_strings).lower()
                sub_genres_.append(value)
        current_key_ = list(dict_.keys())[j]
        dict_[current_key_] = sub_genres_
        j+=1    
    dict_.pop('avant-garde')
    return dict_

In [156]:
dict_ = get_music_genres('https://en.wikipedia.org/wiki/List_of_music_styles')

[<span class="toctext">African</span>, <span class="toctext">Arabic music</span>, <span class="toctext">Asian</span>, <span class="toctext">East Asian</span>, <span class="toctext">South and southeast Asian</span>, <span class="toctext">Avant-garde</span>, <span class="toctext">Blues</span>, <span class="toctext">Caribbean and Caribbean-influenced</span>, <span class="toctext">Comedy</span>, <span class="toctext">Country</span>, <span class="toctext">Easy listening</span>, <span class="toctext">Electronic music</span>, <span class="toctext">Folk</span>, <span class="toctext">Hip hop</span>, <span class="toctext">Jazz</span>, <span class="toctext">Latin</span>, <span class="toctext">Pop</span>, <span class="toctext">R&amp;B and soul</span>, <span class="toctext">Rock</span>, <span class="toctext">Classical music</span>, <span class="toctext">Other</span>, <span class="toctext">References</span>, <span class="toctext">External links</span>, <span class="toctext">Bibliography</span>, <spa

In [157]:
dict_

{'blues': ['african blues',
  'blues rock',
  'blues shouter',
  'british blues',
  'canadian blues',
  'chicago blues',
  'classic female blues',
  'contemporary r&b',
  'country blues',
  'delta blues',
  'detroit blues',
  'electric blues',
  'funk blues',
  'gospel blues',
  'hill country blues',
  'hokum blues',
  'jump blues',
  'kansas city blues',
  'louisiana blues',
  'memphis blues',
  'piedmont blues',
  'punk blues',
  'rhythm and blues',
  'soul blues',
  'st. louis blues',
  'swamp blues',
  'texas blues',
  'west coast blues'],
 'caribbean and caribbean-influenced': ['baithak gana',
  'dancehall music',
  'bouyon',
  'cadence-lypso',
  'calypso',
  'cha-cha-chá',
  'chutney',
  'chutney soca',
  'chutney parang',
  'compas',
  'mambo',
  'mento',
  'merengue',
  'méringue',
  'mozambique',
  'pichakaree',
  'punta',
  'punta rock',
  'rasin',
  'reggae',
  'dub',
  'lovers rock',
  'ragga',
  'ragga jungle',
  'reggae fusion',
  'rocksteady',
  'rumba',
  'ska',
  '2 to

In [158]:
#Create a dataframe from dictionary
genre_class_df = pd.DataFrame(dict_.items(), columns=['genre', 'sub-genres']) 

In [159]:
#Export dataframe to json file
genre_class_df.to_json("music_genres_classification.json")